In [2]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra
using JSON

In [131]:
config = JSON.parsefile("config.json")

#existing_distance = Matrix(CSV.read("inputs/candidate_existing_distance.csv", DataFrame, header = true))
existing_distance_raw = Matrix(CSV.read("inputs/standardized_existing_distance.csv", DataFrame, header = true))
existing_distance1 = existing_distance_raw[:,2:14]
#attraction_distance = Matrix(CSV.read("inputs/candidate_attraction_distance.csv", DataFrame, header = true))
attraction_distance_raw = Matrix(CSV.read("inputs/standardized_attractions_distance.csv", DataFrame, header = true))
attraction_distance = attraction_distance_raw[:,2:23]
demand_raw = Matrix(CSV.read("inputs/standardized_salem_demand.csv", DataFrame, header = true))
demand = demand_raw[:, 17]
attraction_values_raw = Matrix(CSV.read("inputs/standardized_attractions_weights.csv", DataFrame, header = true))
attraction_values = attraction_values_raw[:,6]
candidate_addresses_raw = Matrix(CSV.read("inputs/candidate_address.csv", DataFrame, header = true));


In [ ]:
#raw = Matrix(CSV.read("inputs/candidate_existing_distance.csv", DataFrame, header = false))
#raw_sorted = raw[:,sortperm(raw[1,:])] 
#raw_strings = raw_sorted[2:110,:]
#existing_distance = parse.(Int64,string.(raw_strings))
attraction_distance

In [44]:
existing_distance_raw
candidate_addresses_raw[91,:]

4-element Vector{Any}:
    "24 Fort Ave, Salem, MA 01970, USA"
    "(42.5251058, -70.8774911)"
  42.5251058
 -70.8774911

In [ ]:
demand_raw

In [132]:
#remove stations inside gas company

existing_distance2 = existing_distance1[1:end .!= 80,:] 
#existing_distance = existing_distance[1:end .!= 83,:]
existing_distance = existing_distance2[1:end .!= 91,:]

attraction_distance = attraction_distance[1:end .!= 80,:] 
attraction_distance = attraction_distance[1:end .!= 91,:] 

candidate_addresses_raw1 = candidate_addresses_raw[1:end .!= 80,:] 
candidate_addresses_raw2 = candidate_addresses_raw1[1:end .!= 91,:] 

107×4 Matrix{Any}:
 "90 Willson St, Salem, MA 01970, USA"                           …  42.5074  -70.9075
 "29 Horton St, Salem, MA 01970, USA"                               42.5077  -70.9046
 "2 Laurent Rd, Salem, MA 01970, USA"                               42.5076  -70.9025
 "13 Brooks St, Salem, MA 01970, USA"                               42.5076  -70.8996
 "3 Broadway, Salem, MA 01970, USA"                                 42.5075  -70.897
 "79 Ocean Ave, Salem, MA 01970, USA"                            …  42.5078  -70.8943
 "292 Lafayette St, Salem, MA 01970, USA"                           42.5078  -70.8917
 "10 Cliff St, Salem, MA 01970, USA"                                42.5079  -70.8891
 "32 Shore Ave, Salem, MA 01970, USA"                               42.508   -70.8867
 "19 Greenway Rd, Salem, MA 01970, USA"                             42.5096  -70.9077
 "34 Horton St, Salem, MA 01970, USA"                            …  42.5088  -70.9036
 "192 Jefferson Ave, Salem, MA 01970

In [133]:
row_num =  zeros(size(candidate_addresses_raw2,1), 1)
for i=1:size(candidate_addresses_raw2,1)
    row_num[i] = i
end
row_num
candidate_addresses = [row_num candidate_addresses_raw2]
candidate_addresses

107×5 Matrix{Any}:
   1.0  …  42.5074  -70.9075
   2.0     42.5077  -70.9046
   3.0     42.5076  -70.9025
   4.0     42.5076  -70.8996
   5.0     42.5075  -70.897
   6.0  …  42.5078  -70.8943
   7.0     42.5078  -70.8917
   8.0     42.5079  -70.8891
   9.0     42.508   -70.8867
  10.0     42.5096  -70.9077
  11.0  …  42.5088  -70.9036
  12.0     42.5094  -70.9016
  13.0     42.5099  -70.8996
   ⋮    ⋱           
  96.0  …  42.5274  -70.8897
  97.0     42.5279  -70.881
  98.0     42.5278  -70.879
  99.0     42.529   -70.9081
 100.0     42.5291  -70.9056
 101.0  …  42.5293  -70.9028
 102.0     42.5291  -70.9001
 103.0     42.5295  -70.8921
 104.0     42.5295  -70.8895
 105.0     42.5296  -70.8869
 106.0  …  42.5294  -70.8813
 107.0     42.5297  -70.8791

In [ ]:
demand

In [ ]:
existing_distance

In [ ]:
attraction_distance

In [ ]:
attraction_values

In [134]:
# parameters

N_CANDIDATE_STATIONS = size(existing_distance)[1]
N_NEW_STATIONS = config["N_NEW_STATIONS"]
#DISTANCE_THRESHOLD = config["DISTANCE_THRESHOLD"]
N_CURRENT_STATIONS = size(demand)[1]
N_ATTRACTIONS = size(attraction_distance)[2]

DISTANCE_THRESHOLD = 0.2006853446636252 #need to normalize the 800 meteres value to fit our data

0.2006853446636252

In [ ]:
###FIRST VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# @variable(model, x[i=1:50, j=1:15] >= 0) # amount of waste transported
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# @constraint(model, [i=1:50], sum(x[i, j] for j in 1:15) == q[i])
# @constraint(model, [i=1:50, j=1:15], x[i, j] <= M * z[j])

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], distance[i, j] >=  1000 * z[i])

@objective(model, Max, sum(-distance[i, j] * demand[j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)


println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

In [ ]:
candidate_station_addresses[findall(x -> x==1, stations_chosen)]

In [ ]:
findall(stations_chosen)

In [ ]:
###SECOND VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE

@objective(model, Max, 
    sum(-existing_distance[i, j] * z[i] * demand[j] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    sum(-attraction_distance[i, k] * z[i] * attraction_values[k] * 0 for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

In [ ]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

In [ ]:
###THIRD VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0#0.0000004*6
beta = 0#6
gamma = 0#0.0000012
kappa = 1#1

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE

@objective(model, Max, 
    alpha * sum(-existing_distance[i, j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    beta * sum(demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) + #enforce that demand for stations further away from candidate stations count less
    gamma * sum(-attraction_distance[i, k] * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa * sum(attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

In [ ]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

**OPTIMIZATION FORMULATION:**

Sets:
- candidate stations $i\in \mathcal{I}=\{1,\ldots,n\}$
- existing stations $j\in \mathcal{J}=\{1,\ldots,m\}$
- attractions $k\in \mathcal{K}=\{1,\ldots,q\}$

Decision variable: $z_{i} $ = 1 if candidate station i is built, 0 otherwise

Parameters:
- $e_{ij} $ = distance between candidate station i and existing station j
- $a_{ik} $ = distance between candidate station i and attraction k
- $d_{j} $ = demand at existing station j
- $v_{k} $ = number of google reviews for attraction k
- all parameters are normalized to be between 0 and 1 
    - add epsilon in the denominator to account for distance = 0

Hyperparameters:
- Weights for each term:
    - alpha = weight on distance between candidate station i and existing station j
    - beta = weight on demand at existing station j
    - gamma = weight on distance between candidate station i and attraction k
    - kappa = weight on number of google reviews for attraction k
- Logistic Function:
    - We apply a logistic function to both the demand and google reviews terms
    - This gives more weight to demand/reviews for stations/attractions that are closer to the candidates
    - and less weight to the demand/reviews for stations/attractions that are further from the candidates
    - $\frac{L}{1+e^{-w(distance-q)}} $
    - L = the curve's maximum value
    - w = steepness of the curve
    - q = the distance value of the sigmoid midpoint

$$ 
\max \quad 
\alpha(\sum_{i=1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}+\epsilon} * z_i) +
\beta(\sum_{i=1}^{n} \sum_{j=1}^{m} d_{j}z_i*\frac{L}{1+e^{-w(e_{ij}-q)}}) +
\gamma(\sum_{i=1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}+\epsilon} * z_i) +
\kappa(\sum_{i=1}^{n} \sum_{k=1}^{q} v_{k}z_i*\frac{L}{1+e^{-w(a_{ik}-q)}})
\\

\text{s.t.} \quad \sum_{i = 1}^{n} z_i <= 5\\

e_{ij} >= 0.1z_i \quad \forall i,j\\

z_i\in\{0,1\}

$$



\begin{equation*}
\begin{split}
\max_z & \quad \alpha(\sum_{i=1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}+\epsilon}\ z_i)+ \beta(\sum_{i=1}^{n} \sum_{j=1}^{m} d_jz_i\frac{L}{1+e^{-w(e_{ij}-q)}}) + \gamma(\sum_{i=1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}+\epsilon}z_i) + \kappa(\sum_{i=1}^{n} \sum_{k=1}^{q}v_kz_i \frac{L}{1+e^{-w(a_{ik}-q)}}) \\
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\text{s.t.} & \quad \sum_{i=1}^{n}z_i \leq 5 \quad (1)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad e_{ij} \geq 0.1z_i \quad \forall i,j \quad (2)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad z_i\in\{0,1\} \quad (3)
\end{split}
\end{equation*}

In [141]:
###FOURTH VERSION OF MODEL

using Random

Random.seed!(15093)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0.468#0.5
beta = 7.8#8
gamma = 0.000003
kappa = 2.5

# z: binary variable to indicate whether or not station is selected
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) 

#can add up to 5 new stations
@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) <= N_NEW_STATIONS) #can add up to N new stations

# add constraint that two candidate stations cannot be too close to existing stations (400 meters)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) 

@objective(model, Max, 
    alpha * sum((1/(existing_distance[i, j]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    beta * sum(demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) + 
    gamma * sum((1/(attraction_distance[i, k]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa * sum(attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
Total Cost: 422.0


In [142]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
  48.0
  54.0
  86.0
  96.0
 104.0

In [143]:
candidate_addresses_raw[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "20 Harbor St, Salem, MA 01970, USA"
 "15 Chestnut St, Salem, MA 01970, USA"
 "33 Williams St, Salem, MA 01970, USA"
 "Furlong Park, Franklin St, Salem, MA 01970, USA"
 "5 Southwick St, Salem, MA 01970, USA"

In [ ]:
candidate_addresses[104,:]

**OPTIMIZATION FORMULATION:**

Same as above except we add robustness to account for uncertainty on demand and Google reviews.

We use the box uncertainty set for both terms.

Additional Parameters:
- $t $ = demand term
- $s $ = Google reviews term
- $g_i $ = used to linearize the L1 norm in the regularization term for demand uncertainty
- $h_i $ = used to linearize the L1 norm in the regularization term for Google reviews uncertainty

Additional Hyperparameters:
- $\rho $ = choice of uncertainty set
    - we use the same value of $\rho $ for each term
    - both demand and Google reviews were normalized to be between 0 and 1
    
Constraint 3: Robust counterpart for demand using box uncertainty set

Constraints 4 and 5: Linearize term inside L1 norm for Gurobi

Constraint 6: Robust counterpart for Google reviews using box uncertainty set

Constraints 7 and 8: Linearize term inside L1 norm for Gurobi

# ROBUST

\begin{equation*}
\begin{split}
\max_z & \quad \alpha(\sum_{i=1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}+\epsilon}\ z_i)+ \beta(t) + \gamma(\sum_{i=1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}+\epsilon}z_i) + \kappa(s) \\
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\text{s.t.} & \quad \sum_{i=1}^{n}z_i \leq 5 \quad (1)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad e_{ij} \geq 0.1z_i \quad \forall i,j \quad (2)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad-t \geq (\sum_{i=1}^{n} \sum_{j=1}^{m} -d_jz_i\frac{L}{1+e^{-w(e_{ij}-q)}}) + \rho \sum_{i=1}^{n} g_i \quad (3)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad g_i \geq z_i \frac{L}{1+e^{-w(e_{ij}-q)}} \quad (4)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad g_i \geq -z_i \frac{L}{1+e^{-w(e_{ij}-q)}} \quad (5)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad -s \geq (\sum_{i=1}^{n} \sum_{k=1}^{q} -v_kz_i\frac{L}{1+e^{-w(a_{ik}-q)}}) + \rho \sum_{i=1}^{n} h_i \quad (6)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad h_i \geq z_i \frac{L}{1+e^{-w(a_{ik}-q)}} \quad (7)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad h_i \geq -z_i \frac{L}{1+e^{-w(a_{ik}-q)}} \quad (8)
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\quad z_i\in\{0,1\} \quad (9)
\end{split}
\end{equation*}

In [129]:
###ROBUST VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0.468
beta = 7.8
gamma = 0.000003
kappa = 2.5

#robust
rho1 = 3
rho2 = 3

# z: binary variable to indicate whether or not station is selected
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) 
@variable(model, t)
@variable(model, g[i=1:N_CANDIDATE_STATIONS])
@variable(model, s)
@variable(model, h[i=1:N_CANDIDATE_STATIONS])

#can add up to 5 new stations
@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) <= N_NEW_STATIONS) 

# add constraint that two candidate stations cannot be too close to existing stations (400 meters)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) 

##robust constraints

#demand
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], sum(-demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) 
        for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS)+ (rho1 * sum(g[i])) <= -t) 
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], g[i] >= z[i]*(L/(1+exp(-w*(existing_distance[i,j]-q)))))
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], g[i] >= -z[i]*(L/(1+exp(-w*(existing_distance[i,j]-q)))))

#attraction value
@constraint(model, [i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS], sum(-attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) 
        for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS)+ (rho2 * sum(h[i])) <= -s) 
@constraint(model, [i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS], h[i] >= z[i]*(L/(1+exp(-w*(attraction_distance[i,k]-q)))))
@constraint(model, [i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS], h[i] >= -z[i]*(L/(1+exp(-w*(attraction_distance[i,k]-q)))))

@objective(model, Max, 
    alpha * sum((1/(existing_distance[i, j]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    beta*(t) +
    gamma * sum((1/(attraction_distance[i, k]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa*(s))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Optimal stations: [-0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 1.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 0.0, 0.0]
Total Cost: 397.0


In [130]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
  48.0
  54.0
  80.0
  87.0
 106.0

In [ ]:
48
54
80
87
106